* *Импортируем необходимые библиотеки*

In [34]:
import cv2 as cv
import numpy as np
import glob
np.random.seed(0)

* *Функция для сохранения картинки*

In [35]:
def SaveImage(img,name,dst=''):
    cv.imwrite(dst+name,img)

* *Выявление повреждённой части листа*
  * Для показа следующей картинки нужно нажать любую клавишу

In [36]:
def FindDamagedAndNonDamagedZones(image,name,dst,p1=1,p2=1,manually=False,markers_man=None):

    hsv_image=cv.cvtColor(image,cv.COLOR_BGR2HSV)# Конвертируем изображение в HSV формат
   
    healty_part=cv.inRange(hsv_image,(36,10,25),(86,255,255)) # Выделяем здоровую область
    markers_visualization=np.zeros_like(image,np.uint8)
    markers=np.zeros( (image.shape[0],image.shape[1]), dtype = "int32")
    if manually==False:
        background=cv.inRange(hsv_image,(90,5,25),(240,255,255))+cv.inRange(hsv_image,(0,5,31),(2,255,255)) # Выделяем фон
        shadows=cv.inRange(hsv_image,(0,0,0),(240,255,25)) # Выделяем тень

        
        #Делаем случайную маску с шансами p1 и p2  для того чтобы доавлять маркеры случайно
        healty_randomized_mask=np.random.choice(a=[True, False], size=(image.shape[0],image.shape[1]), p=[p1, 1-p1])  
        background_randomized_mask=np.random.choice(a=[True, False], size=(image.shape[0],image.shape[1]), p=[p2, 1-p2]) 

        #healty_mask=healty_part and healty_randomized_mask
        healty_mask=np.all([healty_part>0,healty_randomized_mask==True],axis=0)

        #background_mask=(shadows or background) and background_randomized_mask
        background_mask=np.any([background>0,shadows>0],axis=0)
        background_mask=np.all([background_mask,background_randomized_mask==True],axis=0)

        #Рисуем картинку с маркерами
        
        markers_visualization[healty_mask==True]=(0,255,128)
        markers_visualization[background_mask==True]=(255,168,211)

       
        markers[healty_mask==True]=255
        markers[background_mask==True]=1
    else:
        markers=markers_man
        markers_visualization[markers==255]=(0,255,128)
        markers_visualization[markers==1]=(255,168,211)
    
    SaveImage(np.concatenate([image,markers_visualization],axis=1),name,dst+'markers/')
    

    #Делаем маркеры
   

    #Запускаем алгоритм
    labels =cv.watershed(hsv_image,markers)
    colors=[(125,0,255),(0,0,0),(125,255,0),(0,255,255)]
   
    result_img=np.zeros_like(image,np.uint8)
    i=0
    for label in np.unique(labels):
        if label == 0:
            continue
        result_img[labels == label] = colors[i]
        i+=1

    watershed_img= np.copy(result_img)
    SaveImage(np.concatenate([image,watershed_img],axis=1),name,dst+'watershed/')

    #Находим больную часть
    ill_part=labels-healty_part
    result_img[ill_part>1]=colors[3]
    SaveImage(np.concatenate([image,result_img],axis=1),name,dst+'recognizing/')

    #Сохраняем и показываем все этапы картинки
    conc=np.concatenate([image,markers_visualization,watershed_img,result_img],axis=1)
    SaveImage(conc,name,dst+'summary/')
    cv.imshow('Display window',conc)
    cv.waitKey(0)
    return result_img




* *Полная обработка всех изображений для разных методов обработки шума*

In [37]:
def ProcessImagesBileteral(images_paths,d,sigmaColor,sigaSpace,p1,p2):
    for image_path in images_paths:
        name=image_path[image_path.rfind('\\')+1:]
        image=cv.imread(image_path)
        
        dst='method/bileteral/'+'d='+str(d)+' sigmaColor='+str(sigmaColor)+' sigmaSpace='+str(sigaSpace)+' markers='+str(p1)+','+str(p2)+'/'
        
        bileteral_img=cv.bilateralFilter(image, d ,  sigmaColor , sigaSpace)
        SaveImage(np.concatenate([image,bileteral_img],axis=1),name,dst+'denoising/')
       
        res=FindDamagedAndNonDamagedZones(bileteral_img,name,dst,p1,p2)
        SaveImage(np.concatenate([image,res],axis=1),name,dst+'comparison/')

def ProcessImagesErode(images_paths,figure,sizes,p1,p2):
    for image_path in images_paths:
        name=image_path[image_path.rfind('\\')+1:]
        image=cv.imread(image_path) 

        dst='method/erosion/Ellipse'+str(sizes)+' markers='+str(p1)+','+str(p2)+'/'

        kernel=cv.getStructuringElement(figure,sizes)
        erode_img=cv.erode(image,kernel)
        SaveImage(np.concatenate([image,erode_img],axis=1),name,dst+'denoising/')
        
        res=FindDamagedAndNonDamagedZones(erode_img,name,dst,p1,p2)
        SaveImage(np.concatenate([image,res],axis=1),name,dst+'comparison/')


def ProcessImagesNonLocalMeans(images_paths,h,hColor,templateWindowSize,searchWindowSize,p1,p2):
    for image_path in images_paths:
        name=image_path[image_path.rfind('\\')+1:]
        image=cv.imread(image_path)

        dst='method/non-local means/'+'h='+str(h)+' hColor='+str(hColor)+' templateWindowSize='+str(templateWindowSize)+' searchWindowSize='+str(searchWindowSize)+' markers='+str(p1)+','+str(p2)+'/'

        nonlm_img=cv.fastNlMeansDenoisingColored(image,None,h,hColor,templateWindowSize,searchWindowSize)
        SaveImage(np.concatenate([image,nonlm_img],axis=1),name,dst+'denoising/')
        
        res=FindDamagedAndNonDamagedZones(nonlm_img,name,dst,p1,p2)

        SaveImage(np.concatenate([image,res],axis=1),name,dst+'comparison/')

* *Получение путей всех картинок и запуск их обработки*

In [38]:
images_paths=[path for path in glob.glob('test/*.jpg')]
ProcessImagesBileteral(images_paths,15,75,75,1,1)
ProcessImagesBileteral(images_paths,15,75,75,0.5,0.5)
ProcessImagesBileteral(images_paths,15,75,75,1,0.2)
ProcessImagesBileteral(images_paths,10,50,50,1,0.2)
ProcessImagesBileteral(images_paths,20,100,100,1,0.2)
ProcessImagesErode(images_paths,cv.MORPH_ELLIPSE,(3,3),1,0.2)
ProcessImagesErode(images_paths,cv.MORPH_ELLIPSE,(5,5),1,0.2)
ProcessImagesErode(images_paths,cv.MORPH_ELLIPSE,(7,7),1,0.2)
ProcessImagesErode(images_paths,cv.MORPH_ELLIPSE,(9,9),1,0.2)
ProcessImagesNonLocalMeans(images_paths,5,10,7,21,1,0.2)
ProcessImagesNonLocalMeans(images_paths,10,10,7,21,1,0.2)
ProcessImagesNonLocalMeans(images_paths,15,10,7,21,1,0.2)


image_path='test/3.jpg'
name=image_path[image_path.rfind('/')+1:]
image=cv.imread(image_path)
      
bileteral_img=cv.bilateralFilter(image, 15 ,  75 , 75)
SaveImage(np.concatenate([image,bileteral_img],axis=1),name,'method/bileteral/manually/denoising/')

markers=np.zeros( (image.shape[0],image.shape[1]), dtype = "int32")
markers[100:180,80:160]=255
markers[10:30,10:30]=1
markers[210:230,10:30]=1
markers[10:30,210:230]=1
markers[210:230,210:230]=1

FindDamagedAndNonDamagedZones(bileteral_img,name,'method/bileteral/manually/',1,0.2,True,markers)
cv.destroyAllWindows()